<a href="https://colab.research.google.com/github/DavidPuigV/Nuclio/blob/main/rnn_textgeneration_keras_gru_quijote_ds1020str.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

## Generación de texto mediante redes neuronales recurrentes

Un ejemplo montado en Keras para RNN usando una GRU con un texto de Quijote

# 1. Librerias

In [48]:
from __future__ import absolute_import, division, print_function
from tensorflow import keras as ks
import tensorflow as tf

import numpy as np
import os
import time

!pip install unidecode
import unidecode

# 2. Cargamos los datos - el corpus basado en textos de Shakespeare

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preparamos los datos haciendo algunas manipulaciones, como la primera, decodificar el texto que viene en Unicode

In [50]:
def preproceso(text):
  text = unidecode.unidecode(text)
  text = text.replace('\n', ' ').replace('\ufeff', '').lower() # nos cargamos saltos y lo pasamos todo a minuscula
  text = ''.join(x for x in text if x not in "%&$#=<>/*+@][")  # nos cargamos caracteres extranos
  while len(text) != len(text.replace("  ", " ")):  # aqui hacemos un replace de muchos espacios
    text = text.replace("  ", " ")
  return text


In [51]:
with open("./quijote.txt", 'r') as infile:
  text = preproceso(infile.read())   # aplicamos el preproceso y limpiamos el texto, vemos longitud y vocabulario de 48 caracteres unicos

print ('Longitud del corpus: {} caracteres'.format(len(text)))
print ('Ejemplo de texto...')
print(text[:250])
vocab = sorted(set(text))

print(vocab)

print ('{} caracteres unicos'.format(len(vocab)))

Longitud del corpus: 2106939 caracteres
Ejemplo de texto...
el ingenioso hidalgo don quijote de la mancha tasa yo, juan gallo de andrada, escribano de camara del rey nuestro senor, de los que residen en su consejo, certifico y doy fe que, habiendo visto por los senores del un libro intitulado el ingenioso hid
[' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
48 caracteres unicos


# 3. Pre-proceso de los datos

## 3.1 Indexado de carácteres
Montamos un indice para los carácteres, para tener valores numéricos, y vemos un ejemplo

In [52]:
char2idx = {u:i for i, u in enumerate(vocab)}    # caracteres a indices y hacemos la inversa
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [53]:
print('{')   # esto es lo que hemos hecho transformando letras por numeros
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
print ('{} ---- carácteres mapeados a números enteros ---- > {}'.format(repr(text[:13]), text_as_int[:13]))   # asi queda el ingenioso

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  "'" :   3,
  '(' :   4,
  ')' :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '0' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ':' :  19,
  ...
}
'el ingenioso ' ---- carácteres mapeados a números enteros ---- > [26 33  0 30 35 28 26 35 30 36 40 36  0]


Fijamos cada texto en secuencias de 100 carácteres, y mostramos como se va a ir entregando la inforamción a la red neuronal recurrente...

In [54]:
seq_length = 100     # aprenderemos de secuencias de 100 caracteres, aqui lo tenemos como un array
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

print(char_dataset)

e
l
 
i
n
g
e
n
i
o
<TensorSliceDataset shapes: (), types: tf.int64>


## 3.2 Definimos las secuencias

In [55]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)   # aqui tenemos las primeras 5 secuancias para entrenar mi red neuronal

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))     

'el ingenioso hidalgo don quijote de la mancha tasa yo, juan gallo de andrada, escribano de camara del'
' rey nuestro senor, de los que residen en su consejo, certifico y doy fe que, habiendo visto por los '
'senores del un libro intitulado el ingenioso hidalgo de la mancha, compuesto por miguel de cervantes '
'saavedra, tasaron cada pliego del dicho libro a tres maravedis y medio; el cual tiene ochenta y tres '
'pliegos, que al dicho precio monta el dicho libro docientos y noventa maravedis y medio, en que se ha'


In [56]:
def split_input_target(chunk):    # las decalamos 1 caracter por delante y por detras
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [57]:
for input_example, target_example in  dataset.take(1):       # las decalamos 1 caracter por delante y por detras
  print ('Secuencia de entrada: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Secuencia de salida:  ', repr(''.join(idx2char[target_example.numpy()])))

Secuencia de entrada:  'el ingenioso hidalgo don quijote de la mancha tasa yo, juan gallo de andrada, escribano de camara de'
Secuencia de salida:    'l ingenioso hidalgo don quijote de la mancha tasa yo, juan gallo de andrada, escribano de camara del'


In [59]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):   # aqui tenemos los pasos, estados de la red neuronal
    print("Paso {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  output esperado: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Paso    0
  input: 26 ('e')
  output esperado: 33 ('l')
Paso    1
  input: 33 ('l')
  output esperado: 0 (' ')
Paso    2
  input: 0 (' ')
  output esperado: 30 ('i')
Paso    3
  input: 30 ('i')
  output esperado: 35 ('n')
Paso    4
  input: 35 ('n')
  output esperado: 28 ('g')


## 3.3 Montamos el dataset

In [60]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# 4. Montamos la red neuronal recurrente

In [61]:
vocab_size = len(vocab)
embedding_dim = 256     # longitud del vector
rnn_units = 1024     # 1024 unidaddes de GRU

In [62]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = ks.Sequential()
    model.add(ks.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]))
    model.add(ks.layers.GRU(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True))
    model.add(ks.layers.Dense(vocab_size))  # nos da probabilidades
    
    return model

In [63]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [64]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 48)            49200     
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


# 5. Cogemos el modelo con pesos aleatorios que hemos creado y generamos una primera predicción con el mismo

In [65]:
for input_example_batch, target_example_batch in dataset.take(1):     # montamos los tensores de calculo para meter dentro la red neuronal
    print(input_example_batch)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[21 38 42 ... 37 22 39]
 [ 0 38 42 ...  6  0 37]
 [40 36  6 ... 47  0 37]
 ...
 [26 39 36 ... 38 42 26]
 [22  0 33 ... 33 22 36]
 [41 36 39 ...  3  3  0]], shape=(64, 100), dtype=int64)
(64, 100, 48) # (batch_size, sequence_length, vocab_size)


In [66]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [67]:
sampled_indices

array([44, 26, 15, 12,  0, 25, 40,  0, 45, 18, 20, 14, 20, 14, 26, 39, 22,
       21, 11, 11, 45, 26,  5, 12,  9, 43, 41, 20, 35, 44, 12, 10,  2, 34,
        3, 45, 47, 27,  1, 13, 47, 34, 36,  5, 19,  5, 10, 17, 36, 19,  1,
       29, 10, 18,  3, 10,  2, 15, 41, 18, 43, 23, 30, 16,  1, 37, 40, 15,
       47, 33, 13,  9, 26, 44,  4, 45, 34, 42, 43, 26, 32, 22, 32, 43,  4,
       12, 21, 20, 13, 16, 12, 36,  7,  9,  9, 31,  5, 12, 44, 46])

In [68]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '?que locura es esta? mire que no hay gigante ni caballero alguno, ni gatos, ni armas, ni escudos par'

Next Char Predictions: 
 'we63 ds x9;5;5era?22xe)30vt;nw31"m\'xzf!4zmo):)18o:!h19\'1"6t9vbi7!ps6zl40ew(xmuvekakv(3?;473o-00j)3wy'


# 6. Creamos una funcion de perdida loss

La función de pérdida estándar <code>tf.keras.losses.sparse_categorical_crossentropy</code> funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits, necesitamos establecer la <code>from_logits</code>. Logits son un conjunto de probabilidades sin scalar, es decir, que no hay que meter ningún "softmax" en la salida de la red neuronal.

Aprovechamos y calculamos el error escalar que estamos cometiendo en la predicción que hemos hecho en la anterior etapa

In [69]:
def loss(labels, logits):     # definimos la funcion de perdidas
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 48)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.8740597


# 7. Compilamos el modelo

In [70]:
model.compile(
    optimizer = "adam",
    loss = loss,
    metrics = ['accuracy'])

# 8. Definimos checkpoints donde almacenar los modelos a cada epoch, usando callbacks

In [71]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=ks.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# 9. Entrenamos el modelo 

In [72]:
EPOCHS=10
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/10
329/329 [==============================] - 27s 76ms/step - loss: 2.5165 - accuracy: 0.2890
Epoch 2/10
329/329 [==============================] - 26s 78ms/step - loss: 1.5936 - accuracy: 0.4984
Epoch 3/10
329/329 [==============================] - 26s 78ms/step - loss: 1.3600 - accuracy: 0.5687
Epoch 4/10
329/329 [==============================] - 26s 79ms/step - loss: 1.2676 - accuracy: 0.5947
Epoch 5/10
329/329 [==============================] - 26s 80ms/step - loss: 1.2125 - accuracy: 0.6104
Epoch 6/10
329/329 [==============================] - 26s 80ms/step - loss: 1.1761 - accuracy: 0.6202
Epoch 7/10
329/329 [==============================] - 26s 80ms/step - loss: 1.1434 - accuracy: 0.6301
Epoch 8/10
329/329 [==============================] - 26s 80ms/step - loss: 1.1151 - accuracy: 0.6389
Epoch 9/10
329/329 [==============================] - 26s 79ms/step - loss: 1.0873 - accuracy: 0.6470
Epoch 10/10
329/329 [==============================] - 26s 79ms/step - loss: 1.062

# 10. Reconstruimos el modelo con los pesos entrenados

Debido a la forma en que el estado RNN se pasa de un paso de tiempo a otro, el modelo solo acepta un tamaño de batch fijo una vez construido.

Para ejecutar el modelo con un batch_size diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el checkpoint.


In [73]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 48)             49200     
Total params: 3,999,792
Trainable params: 3,999,792
Non-trainable params: 0
_________________________________________________________________


# 11.Montamos una función para generar texto

Consiste en que iteremos generando 1000 caracteres a partir de una semilla definida.

Existe un parametro llamado **temperatura** que lo modificaremos para ver los resultados.

In [74]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 0.5
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

      if idx2char[predicted_id] == ",":
        text_generated.append('\n')

  return (start_string + ''.join(text_generated))

# 12. Lanzamos nuestra predicción

In [75]:
print(generate_text(model, start_string=u"dulcinea "))

dulcinea en las salas penas,
 sancho,
 que me diga a vuestra merced a vuestra merced alguna cosa que la hermosura de la hija de aquella belleza de mi cuerpo de la insolencia de su senor,
 y para el cual podia ser que el cielo hizo de la mancha la sangre se vinieron a caballo y a decir mas cuantos lo primero. -eso no -respondio sancho-,
 porque si el cielo me lo ha querido dar a la cabeza y van a un hombre a los caballeros andantes han de saber hacer lo que tanto despues a caballo se le ofrecio a las manos a la caballeriza,
 se quedo de alli a manara de las armas de tan cortesano a la cabeza con las armas,
 las cuales bastaban de pedro a los amantes,
 la deshonestidad y recatada,
 porque a su senor tio de la enfermedad de que sea aquella noche que llegaban compania en las manos,
 como si dijeren en ellos un poco a la venta,
 que estaba en la cabeza en su amo que tantas veces la habia de alabar al buen pario de mano a la senora dulcinea,
 y que le habia puesto la cabeza con las alas del 